In [1]:
#------------------standard packages-----------------
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pickle
import json
from thefuzz import fuzz, process

#-----------------caio's modules---------------------
import sys, os #add code folder to sys.path
sys.path.append(os.path.abspath('../../data_processing'))

from cricsheet_read import cricsheet_read #data processing

#--------------cricket metric functions--------------
import cricket_metrics
from cricket_metrics import *

metrics=[getattr(cricket_metrics,m) for m in dir(cricket_metrics)
         if m not in ['__builtins__','__cached__','__doc__','__file__',
                      '__loader__','__name__','__package__','__spec__',
                      'dt','np','pd','wicket_types']]

#---------------------load data----------------------
match_stack, results = cricsheet_read()

In [13]:
def get_metrics(match_stack,player_list,form=365,aggregate=False):
    
    form_stack=match_stack[match_stack['start_date']>match_stack['start_date'].max()-dt.timedelta(days=form)]
    
    def player_row(player):
        row_dict={}
        
        if player=='all':
            row_dict['player']='all'
            for metric in metrics:
                row_dict[metric.__name__]=metric(match_stack,player_list)
                row_dict[metric.__name__+'_form']=metric(form_stack,player_list)
        else:
            row_dict['player']=player
            for metric in metrics:
                row_dict[metric.__name__]=metric(match_stack,[player])
                row_dict[metric.__name__+'_form']=metric(form_stack,[player])
                
        return row_dict
    if aggregate==True:
        return pd.DataFrame.from_dict([player_row('all')]).drop('player',axis=1)
    else:
        return pd.DataFrame.from_dict([player_row(player) for player in tqdm(player_list+['all'])])

In [17]:
team_h = ['RG Sharma', 'Babar Azam', 'SPD Smith', 'KS Williamson', 'Q de Kock',
          'M Labuschagne', 'JO Holder', 'R Ashwin', 'TA Boult', 'SCJ Broad', 'PJ Cummins']

team_c = ['DA Warner', 'V Kohli', 'BA Stokes', 'C de Grandhomme', 'MS Dhoni',
          'JE Root', 'JDS Neesham', 'MA Starc', 'Rashid Khan', 'JJ Bumrah', 'SL Malinga']

team_harsha = ['JC Buttler', 'Mohammad Rizwan', 'MR Marsh', 'MM Ali', 'GJ Maxwell',
               'AD Russell', 'SP Narine', 'Rashid Khan', 'Shaheen Shah Afridi', 'A Nortje', 'JJ Bumrah']

In [14]:
get_metrics(match_stack,team_h)

  0%|          | 0/12 [00:00<?, ?it/s]

,player,bat_avg,bat_avg_form,bat_sr,bat_sr_form,bowl_avg,bowl_avg_form,bowl_sr,bowl_sr_form,economy,economy_form,rel_bat_avg,rel_bat_avg_form,rel_bat_sr,rel_bat_sr_form,rel_economy,rel_economy_form
0,RG Sharma,31.619318,29.307692,131.188119,127.852349,30.200000,NaN,22.600000,NaN,8.017699,11.142857,131.523450,127.255031,103.113639,101.117108,1.018442,1.001444
1,Babar Azam,41.500000,42.343750,126.892231,129.541109,NaN,NaN,NaN,NaN,NaN,NaN,175.186503,167.705815,98.857757,97.938667,0.988452,0.980873
2,SPD Smith,31.829268,20.777778,128.867676,103.888889,22.041667,NaN,16.333333,NaN,8.096939,NaN,128.694849,81.807975,101.035969,82.531940,1.009286,0.823569
3,KS Williamson,34.057377,29.133333,125.870948,115.000000,25.777778,NaN,19.111111,NaN,8.093023,NaN,138.721852,114.518653,96.063026,92.439490,0.949292,0.921702
4,Q de Kock,31.066667,37.318182,133.000357,126.113671,NaN,NaN,NaN,NaN,NaN,NaN,127.010152,172.062952,103.693934,105.250151,1.031091,1.046548
5,M Labuschagne,32.150000,43.538462,128.600000,134.761905,20.100000,17.842105,13.500000,12.315789,8.933333,8.692308,136.681148,181.662566,98.111125,100.462282,0.970031,0.995671
6,JO Holder,16.890411,16.105263,132.154341,122.400000,28.752212,25.064516,22.442478,18.935484,7.686909,7.942078,82.658703,79.110147,110.862379,103.063918,1.088634,1.008208
7,R Ashwin,11.538462,14.666667,111.386139,104.761905,27.762238,43.428571,24.083916,34.857143,6.916376,7.475410,55.246951,78.086420,90.322174,95.465902,0.896717,0.939389
8,TA Boult,5.125000,NaN,71.929825,70.000000,24.808333,24.392857,18.033333,19.285714,8.254159,7.588889,27.054874,NaN,56.507964,63.281453,0.562453,0.699624
9,SCJ Broad,7.611111,NaN,103.787879,NaN,22.257143,NaN,17.914286,NaN,7.454545,NaN,37.127930,NaN,79.547684,NaN,0.805073,NaN


In [15]:
get_metrics(match_stack,team_c)

  0%|          | 0/12 [00:00<?, ?it/s]

,player,bat_avg,bat_avg_form,bat_sr,bat_sr_form,bowl_avg,bowl_avg_form,bowl_sr,bowl_sr_form,economy,economy_form,rel_bat_avg,rel_bat_avg_form,rel_bat_sr,rel_bat_sr_form,rel_economy,rel_economy_form
0,DA Warner,39.127451,36.769231,140.305853,130.245232,NaN,NaN,NaN,NaN,12.000000,NaN,159.650817,151.425554,110.065380,106.898739,1.100851,1.068113
1,V Kohli,37.680982,28.928571,130.708661,120.178042,92.000000,NaN,62.750000,NaN,8.796813,NaN,146.725875,122.791289,100.021132,95.220940,0.992466,0.944051
2,BA Stokes,24.671053,14.714286,136.462882,113.186813,30.290323,22.625000,20.774194,15.000000,8.748447,9.050000,98.506621,54.980984,102.885636,82.260010,1.021916,0.843444
3,C de Grandhomme,22.619718,14.000000,152.227488,138.842975,50.466667,40.666667,32.133333,27.333333,9.423237,8.926829,95.343187,69.740099,114.426887,118.007937,1.144210,1.199195
4,MS Dhoni,39.550000,16.285714,136.457734,106.542056,NaN,NaN,NaN,NaN,NaN,NaN,154.097089,66.049226,105.380438,80.994671,1.050359,0.807298
5,JE Root,38.638889,24.250000,135.839844,106.593407,24.875000,29.666667,18.000000,22.000000,8.291667,8.090909,159.460296,101.285141,98.634419,81.079483,0.986411,0.849683
6,JDS Neesham,19.508475,17.444444,132.756632,153.921569,30.272727,32.714286,19.675325,22.714286,9.231683,8.641509,83.388401,70.901701,101.929984,111.426889,1.019481,1.142269
7,MA Starc,13.500000,19.666667,102.162162,111.320755,20.320755,34.615385,16.311321,23.076923,7.474841,9.000000,63.937093,101.324229,82.463505,93.561950,0.818344,0.942812
8,Rashid Khan,11.325301,11.695652,141.993958,145.405405,17.601911,17.677966,16.496815,16.305085,6.401931,6.505198,54.827429,54.447640,115.540105,116.284050,1.165802,1.176367
9,JJ Bumrah,14.000000,8.000000,100.000000,94.117647,23.308943,19.523810,18.910569,15.714286,7.395529,7.454545,73.101727,42.976836,82.795923,76.954744,0.830073,0.741059


In [18]:
get_metrics(match_stack,team_harsha)

  0%|          | 0/12 [00:00<?, ?it/s]

,player,bat_avg,bat_avg_form,bat_sr,bat_sr_form,bowl_avg,bowl_avg_form,bowl_sr,bowl_sr_form,economy,economy_form,rel_bat_avg,rel_bat_avg_form,rel_bat_sr,rel_bat_sr_form,rel_economy,rel_economy_form
0,JC Buttler,35.843137,46.941176,148.016194,142.754919,NaN,NaN,NaN,NaN,NaN,NaN,142.261094,185.808824,110.589121,108.370201,1.108358,1.084228
1,Mohammad Rizwan,40.103448,57.517241,126.481784,132.066508,NaN,NaN,NaN,NaN,NaN,NaN,175.712835,243.253071,99.678559,100.984213,0.996978,1.012205
2,MR Marsh,34.120482,43.280000,131.354360,139.432990,27.578947,21.000000,19.736842,16.071429,8.384000,7.840000,151.272015,193.218270,103.836115,107.935136,1.036331,1.084579
3,MM Ali,27.910112,25.826087,153.049908,139.110070,25.512500,19.000000,20.050000,17.736842,7.634663,6.427300,107.153036,103.691981,112.256265,103.545219,1.128577,1.057934
4,GJ Maxwell,28.300448,27.277778,152.919796,139.886040,31.833333,37.071429,24.519608,28.714286,7.789684,7.746269,116.003811,114.758566,115.904087,107.982206,1.166802,1.092448
5,AD Russell,25.662722,23.444444,172.445328,169.251337,27.068493,27.552632,18.369863,17.131579,8.841163,9.649770,108.391320,107.864075,133.298818,129.955213,1.343106,1.323611
6,SP Narine,14.985507,13.692308,149.638205,127.142857,22.297735,20.269231,21.498382,21.230769,6.223092,5.728261,68.066563,61.366923,117.490650,102.737210,1.199653,1.041235
7,Rashid Khan,11.325301,11.695652,141.993958,145.405405,17.601911,17.677966,16.496815,16.305085,6.401931,6.505198,54.827429,54.447640,115.540105,116.284050,1.165802,1.176367
8,Shaheen Shah Afridi,5.142857,6.500000,76.595745,108.333333,22.989796,22.333333,17.867347,17.694444,7.720160,7.572998,26.873720,35.020722,62.580410,85.306888,0.630396,0.822393
9,A Nortje,3.000000,3.000000,48.000000,42.857143,19.865385,15.777778,16.461538,15.481481,7.240654,6.114833,14.663342,16.172507,39.381546,36.888718,0.378705,0.353028


In [ ]:
#--------------------cricket data wrangling-------------------
def cric_wrangle(match_stack,results,hist_data_length,form=30,filename='cricket_data'):
    
    def match_row(match_id):
        row_dict={}
        
        match=match_stack[match_stack['match_id']==match_id]
#---------------------match_info--------------------   
        row_dict['match_id']=match_id
        row_dict['date']=match['start_date'].iloc[0]
        row_dict['set_team']=match['batting_team'].iloc[0]
        row_dict['chase_team']=match['bowling_team'].iloc[0]
        row_dict['set_team_win']=results.loc[results['match_id']==match_id,
                                    'result'].iloc[0]==row_dict['set_team']
        row_dict['set_team_toss']=results.loc[results['match_id']==match_id,
                                    'toss_winner'].iloc[0]==row_dict['set_team']
        row_dict['1_in_wickets']=match[match['innings']==1]['wicket_type'].count()
        row_dict['1w_score']=sum(match['wicket_type'].notna()*(match['innings']==1)*match['n_balls'])
#------------filter for players in teams------------ 
        batters=match.loc[match['batting_team']==match['batting_team'].iloc[0],'striker']
        bowlers=match.loc[match['bowling_team']==match['batting_team'].iloc[0],'bowler']
        set_team=list(np.unique(np.concatenate([batters,bowlers])))

        batters=match.loc[match['batting_team']==match['bowling_team'].iloc[0],'striker']
        bowlers=match.loc[match['bowlingteam']==match['bowling_team'].iloc[0],'bowler']
        chase_team=list(np.unique(np.concatenate([batters,bowlers])))
        
        sub_stack=pd.concat([match_stack[match_stack['striker'].isin(set_team+chase_team)],
                    match_stack[match_stack['bowler'].isin(set_team+chase_team)]]).drop_duplicates()  
#-------filter for games before current match-------
        hist_start_date=row_dict['date']-dt.timedelta(days=365*hist_data_length)
    
        sub_stack=sub_stack[sub_stack['start_date'].between(hist_start_date,row_dict['date'],inclusive='left')] 
#---------------------get metrics-------------------
        row_metrics=get_metrics(sub_stack,set_team,form=form,aggregate=True)-get_metrics(
            sub_stack,chase_team,form=form,aggregate=True)
        
        for metric in row_metrics.columns:
            row_dict[metric]=row_metrics[metric][0]
        
        return row_dict
#-------------------produce dataset-----------------   
    match_list=match_stack['match_id'].unique()
    
    data=pd.DataFrame.from_dict([match_row(match_id) for match_id in tqdm(match_list)])
    
    hist_threshold=data['date'].min()+dt.timedelta(days=366*hist_data_length)
    data=data[data['date']>hist_threshold]
    data=data[~data.isin([np.nan, -np.inf,np.inf]).any(1)]
    
    data.to_csv('../data/'+filename+'.csv')
    return data
#--------------------pepare data for wrangle------------------
[match_stack,results]=cric_read(leagues=['psl'])

data=cric_wrangle(match_stack,results,hist_data_length=2)

In [ ]:
#----------------------------get metrics----------------------------
def get_metrics(match_stack,player_list,form=30,aggregate=False):
    
    form_stack=match_stack[match_stack['start_date']>match_stack['start_date'].max()-dt.timedelta(days=form)]
    
    def player_row(player):
        row_dict={}
        
        if player=='all':
            row_dict['player']='all'
            for metric in metrics:
                row_dict[metric.__name__]=metric(match_stack,player_list)
                row_dict[metric.__name__+'_form']=metric(form_stack,player_list)
        else:
            row_dict['player']=player
            for metric in metrics:
                row_dict[metric.__name__]=metric(match_stack,[player])
                row_dict[metric.__name__+'_form']=metric(form_stack,[player])
                
        return row_dict
    if aggregate==True:
        return pd.DataFrame.from_dict([player_row('all')]).drop('player',axis=1)
    else:
        return pd.DataFrame.from_dict([player_row(player) for player in tqdm(player_list+['all'])])

#-------------------------------test--------------------------------
[match_stack,results]=cric_read()
get_metrics(match_stack,['JM Anderson', 'AD Russell','AB de Villiers'])